fairnessを考慮したデータ表現を得るencoderをneural netを使って構築する

In [1]:
import os
import numpy as np
import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import save_image
from matplotlib import pyplot as plt
%matplotlib inline

cuda = torch.cuda.is_available()
if cuda:
    print('cuda is available!')
    
num_epochs = 20
batch_size = 128
learning_rate = 0.001
FEATURES_NUM=81
A_x=1
A_y=1
A_z=1
out_dir = './result/fair_autoencoder'

ModuleNotFoundError: No module named 'torch'

In [ ]:
class fair_autoencoder(nn.Module):
    
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(FEATURES_NUM, 40),
            nn.ReLU(True),
            nn.Linear(40, 20),
            nn.ReLU(True),
            nn.Linear(20, 10),
            nn.ReLU(True),
            nn.Linear(10, 5),
            nn.Softmax())
        
        self.decoder = nn.Sequential(
            nn.Linear(5, 10),
            nn.ReLU(True),
            nn.Linear(10, 20),
            nn.ReLU(True),
            nn.Linear(20, 40),
            nn.ReLU(True),
            nn.Linear(40, FEATURES_NUM),
            nn.Tanh()
        )
        self.l1 = nn.Linear(5, 2) 

    def forward(self, x):
        z = self.encoder(x)
        xhat = self.decoder(z)
        y=self.l1(z)
        return y,xhat,z

model = fair_autoencoder()
if cuda:
    model.cuda()

In [ ]:
criterion = my_loss_func
optimizer = torch.optim.Adam(model.parameters(),
                             lr=learning_rate,
                             weight_decay=1e-5)

In [ ]:
def my_loss_func(y,labels,x,xhat,z):
    L_x=nn.MSELoss(x,xhat,size_average=False)# 再構成誤差
    L_y=nn.CrossEntropyLoss(y,labels,size_average=False) # 負の対数尤度
    L_z=compute_Lz(z,x) # 公平性制約
    return A_x*L_x+A_y*L_y+A_z*L_z

def compute_Lz(z,x):
    Z_0=[]
    Z_1=[]
    for i in range(x):
        if x[i][FEATURES_NUM-1]==0:
            Z_0.append(z[i])
        else:
            Z_1.append(z[i])
    M_0=torch.mean(Z_0)
    M_1=torch.mean(Z_1)
    Lz=nn.L1Loss(M_1,M_0,size_average=False)
    return Lz